# 카드정보 DB 조회 및 크로마 DB 삽입

In [1]:
from langchain_openai import ChatOpenAI
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()

# 환경 변수 읽기
HOST = os.getenv('DB_HOST')
PORT = os.getenv('PORT')
USER = os.getenv('USER')
PASSWORD = os.getenv('PW')
DATABASE = os.getenv('DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# # 환경 변수 읽기

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain_core.documents import Document
import mysql.connector
from mysql.connector import Error


try:
    # 데이터베이스 연결
    connection = mysql.connector.connect(
        host=HOST,
        database=DATABASE,
        user=USER,
        password=PASSWORD
    )

    if connection.is_connected():
        print("데이터베이스에 성공적으로 연결되었습니다.")

        # 커서 생성
        cursor = connection.cursor()

        # 모든 데이터 조회 쿼리
        query = "SELECT title, excerpt, content FROM kb_card limit 1"

        # 쿼리 실행
        cursor.execute(query)

        # 결과 가져오기
        records = cursor.fetchall()

        # 결과 출력
        print("kb_card 테이블의 데이터:")
        for row in records:
            print(row)

except Error as e:
    print(f"Error: {e}")

finally:
    # 연결 종료
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")

docs = [Document(
        metadata={"title": record[0], "excerpt" : record[1], "language" : "ko"},
        page_content=record[2]
    ) for record in records]

In [2]:
host

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Retrieve and generate using the relevant snippets of the blog.
splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [4]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

: 

In [ ]:
from langchain_core.prompts import PromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """
당신은 KB국민은행에 대한 질문에 답변하는 작업을 돕는 은행원입니다.
다음에 제공된 맥락을 사용하여 질문에 한국말로 답변하십시오.
특히 몇 %를 적립할 수 있으며 할인 받을 수 있는 지 위주로 집중적으로 암기하여 대답하세요.
5문장 이상으로 최대한 자세히 답변해주세요.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

# rag_chain.invoke("빵먹는거 얼마나 할인해주는데?")

In [ ]:
rag_chain.invoke("나 곧 결혼해 카드 추천해줘")

# 스트리밍 적용

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
rag_chain.invoke("트래블러스 카드도 교통카드 후불 가능해? ")

In [ ]:
import time
output = {}
curr_key = None
for chunk in rag_chain_with_source.stream("트래블러스 kb pay 혜택"):
    for key in chunk:
        if key == "answer":
            print(chunk[key], end="")
            if chunk[key] == "." :
                print("")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Retrieve and generate using the relevant snippets of the blog.
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

from langchain_core.prompts import PromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """
당신은 KB국민은행에 대한 질문에 답변하는 작업을 돕는 은행원입니다.
다음에 제공된 맥락을 사용하여 질문에 한국말로 답변하십시오.
특히 몇 %를 적립할 수 있으며 할인 받을 수 있는 지 위주로 집중적으로 암기하여 대답하세요.
5문장 이상으로 최대한 자세히 답변해주세요.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

# rag_chain.invoke("빵먹는거 얼마나 할인해주는데?")

In [ ]:
from IPython.display import Markdown, display
template = """
당신은 KB국민은행에 대한 질문에 답변하는 작업을 돕는 은행원입니다.
다음에 제공된 맥락을 사용하여 질문에 한국말로 답변하십시오.
특히 몇 %를 적립할 수 있으며 할인 받을 수 있는 지 위주로 집중적으로 암기하여 대답하세요.
5문장 이상으로 최대한 자세히 답변해주세요.
만약 추가 정보가 필요하다면 추가 정보를 요구해도 좋습니다.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

# 가계부


In [ ]:
import mysql.connector
from mysql.connector import Error

try:
    # 데이터베이스 연결
    connection = mysql.connector.connect(
        host=HOST,
        database=DATABASE,
        user=USER,
        password=PASSWORD
    )

    if connection.is_connected():
        print("데이터베이스에 성공적으로 연결되었습니다.")

        # 커서 생성
        cursor = connection.cursor()

        # 모든 데이터 조회 쿼리
        query = "SELECT date_format(date, '%Y-%m-%d') as date, category, CAST(amount as SIGNED) FROM transactions where user_id = 5"

        # 쿼리 실행
        cursor.execute(query)

        # 결과 가져오기
        records = cursor.fetchall()

        # 결과 출력
        print("거래내역 데이터:")
        for row in records:
            print(row)

except Error as e:
    print(f"Error: {e}")

finally:
    # 연결 종료
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")


In [ ]:
docs

In [ ]:
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate

docs = [Document(
        page_content=str(record)
    ) for record in records]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# Retrieve and generate using the relevant snippets of the blog.
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """
당신은 KB국민은행에 대한 질문에 답변하는 작업을 돕는 은행원입니다.
다음에 제공된 맥락을 사용하여 질문에 한국말로 답변하십시오.
특히 몇 %를 적립할 수 있으며 할인 받을 수 있는 지 위주로 집중적으로 암기하여 대답하세요.
5문장 이상으로 최대한 자세히 답변해주세요.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("빵먹는거 얼마나 할인해주는데?")

In [ ]:
rag_chain.invoke("난 학생인데 어떤 카드가 좋아?")

In [ ]:
rag_chain.invoke("내가 12월에 총 얼마를 썼지?")